# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Access data files
weather_metadata = "data/my_weather_data.csv"

# Read and load the weather data 
weather_data = pd.read_csv(weather_metadata)

# create a dataframe with weather_data
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Lagoa,37.1363,-8.4541,66.80,68,0,8.03,PT,1569736800
1,1,Rikitea,-23.1203,-134.9692,71.66,86,6,10.51,PF,1569736800
2,2,Albany,42.6512,-73.7550,64.40,95,88,4.76,US,1569736800
3,3,Esperance,-33.8583,121.8932,62.44,87,37,18.97,AU,1569736800
4,4,Faanui,-16.4834,-151.7500,78.16,84,19,20.13,PF,1569736800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

#create humidity rate as float

humidity = weatherdf["Humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
mac_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrow_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrow_city_df


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,Byron Bay,-28.6500,153.6167,71.60,63,0,6.08,AU,1569736800
32,32,Busselton,-33.6445,115.3489,74.41,75,0,3.22,AU,1569736800
56,56,Pombia,35.0089,24.8684,75.87,50,0,7.94,GR,1569736800
76,76,Shahreza,32.0116,51.8652,73.13,15,0,6.53,IR,1569736800
103,103,Miranda,-20.2402,-56.3845,75.46,49,0,5.75,BR,1569736800
226,226,Avera,33.1940,-82.5271,72.52,89,0,3.96,US,1569736800
245,245,Bako,12.2860,-0.7261,75.11,85,0,9.53,BF,1569736800
264,264,Morondava,-20.2954,44.2798,72.36,65,0,2.21,MG,1569736800
268,268,Cayenne,4.9371,-52.3259,77.58,88,0,5.08,GF,1569736800
282,282,Santa Ana,30.5437,-111.1213,76.18,59,0,5.08,MX,1569736800


#### Hotel Map
Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap

In [7]:
# Create a hotel_df
hotel_df = narrow_city_df.loc[:,["City","Country", "Latitude", "Longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
24,Byron Bay,AU,-28.6500,153.6167,
32,Busselton,AU,-33.6445,115.3489,
56,Pombia,GR,35.0089,24.8684,
76,Shahreza,IR,32.0116,51.8652,
103,Miranda,BR,-20.2402,-56.3845,
226,Avera,US,33.1940,-82.5271,
245,Bako,BF,12.2860,-0.7261,
264,Morondava,MG,-20.2954,44.2798,
268,Cayenne,GF,4.9371,-52.3259,
282,Santa Ana,MX,30.5437,-111.1213,


In [8]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 24: Byron Bay.
Missing field/result... skipping.
------------
Retrieving Results for Index 32: Busselton.
Missing field/result... skipping.
------------
Retrieving Results for Index 56: Pombia.
Missing field/result... skipping.
------------
Retrieving Results for Index 76: Shahreza.
Missing field/result... skipping.
------------
Retrieving Results for Index 103: Miranda.
Missing field/result... skipping.
------------
Retrieving Results for Index 226: Avera.
Missing field/result... skipping.
------------
Retrieving Results for Index 245: Bako.
Missing field/result... skipping.
------------
Retrieving Results for Index 264: Morondava.
Missing field/result... skipping.
------------
Retrieving Results for Index 268: Cayenne.
Missing field/result... skipping.
------------
Retrieving Results for Index 282: Santa Ana.
Missing field/result... skipping.
------------
Retrieving Results for Index 330: Villamontes.
Missing field/result... skipping.
------------
Retriev

In [10]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
24,Byron Bay,AU,-28.6500,153.6167,
32,Busselton,AU,-33.6445,115.3489,
56,Pombia,GR,35.0089,24.8684,
76,Shahreza,IR,32.0116,51.8652,
103,Miranda,BR,-20.2402,-56.3845,
226,Avera,US,33.1940,-82.5271,
245,Bako,BF,12.2860,-0.7261,
264,Morondava,MG,-20.2954,44.2798,
268,Cayenne,GF,4.9371,-52.3259,
282,Santa Ana,MX,30.5437,-111.1213,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
plt.savefig("vacationmarkers.png")
fig



Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>